In [36]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00


In [37]:
import os
import re
import streamlit as st
from pprint import pprint
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import chromadb
import openai
from openai import OpenAI
with open('/content/api_key.txt', 'r') as f:
    key = f.read()
    os.environ["OPENAI_API_KEY"] = key
    openai.api_key = key

In [4]:
pdf_file = "{your file path}"
reader = PdfReader(pdf_file)
ipcc_texts = [page.extract_text().strip() for page in reader.pages]

In [5]:
ipcc_texts_filt = ipcc_texts[5: -5]
print(f"Number of pages: {len(ipcc_texts_filt)}")


Number of pages: 46


In [6]:
ipcc_wo_header_footer = [re.sub(r'\d+\nTechnical Summary', '', s) for s in ipcc_texts_filt]
# remove \nTS
ipcc_wo_header_footer = [re.sub(r'\nTS', '', s) for s in ipcc_wo_header_footer]

# remove TS\n
ipcc_wo_header_footer = [re.sub(r'TS\n', '', s) for s in ipcc_wo_header_footer]
ipcc_wo_header_footer


['Indus Valley Ayurvedic Centre\n\xa0\n\xa0\nThe\nCentre\nis\nsituated\nat\nthe\nfoot\nof\nthe\nscenic\nChamundi\nHills\nand\nadjacent\nto\nthe\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\nRoyal\nLalitamahal\nPalace.\nThe\nCentre\nhas\nbeen\ndesigned\non\nthe\nbasis\nof\nancient\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\nprinciples\nof\nVastu,\nthe\nscience\nof\nconstruction\nand\narchitecture.\nConsequently,\nall\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\nstructural\nfeatures\nblend\nharmoniously\nto\nprovide\na\nsublime\nsense\nof\nexperience\nand\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\nharmony.\nFurthermore,\nthe\nfunctional\naspects\nof\nthe\nCentre\nare\nfashioned\nto\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\n\xa0\naccommodate the five elements in a natural way.\n\xa0\nEARTH\nrepresented\nby\nherbs\nused\nfor\nthe\nther

In [7]:
char_splitter = RecursiveCharacterTextSplitter(
    separators= ["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0.2
    )

texts_char_splitted = char_splitter.split_text('\n\n'.join(ipcc_wo_header_footer))
print(f"Number of chunks: {len(texts_char_splitted)}")

Number of chunks: 97


In [8]:
token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=0.2,
    tokens_per_chunk=256
    )

texts_token_splitted = []
for text in texts_char_splitted:
    try:
        texts_token_splitted.extend(token_splitter.split_text(text))
    except:
        print(f"Error in text: {text}")
        continue

print(f"Number of chunks: {len(texts_token_splitted)}")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of chunks: 97


In [9]:
chroma_client = chromadb.PersistentClient(path="/content/db")
chroma_collection = chroma_client.get_or_create_collection("ayurveda")

In [10]:
ids = [str(i) for i in range(len(texts_token_splitted))]
chroma_collection.add(
    ids=ids,
    documents=texts_token_splitted
    )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 12.7MiB/s]


In [11]:
query = "what is the base of ayuerveda sciences, and how can i treat common cold with it?"
res = chroma_collection.query(query_texts=[query], n_results=5)

In [33]:
def rag(query, n_results=5):
    res = chroma_collection.query(query_texts=[query], n_results=n_results)
    docs = res["documents"][0]
    joined_information = ';'.join([f'{doc}' for doc in docs])
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert on ayurvedic knowledge. Your users are asking questions about information contained in attached information."
            "You will be shown the user's question, and the relevant information. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {joined_information}"}
    ]
    openai_client = OpenAI()
    model = "gpt-3.5-turbo"
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content, docs

In [ ]:
query = "what is ayurveda and how can we treat common cold?"

rag(query=query, n_results=5)

In [ ]:
user_query = st.text_input(label="", help="Ask here to learn about basic ayurveda", placeholder="What do you want to know about ayurveda?")

rag_response, raw_docs = rag(user_query)

st.header("Raw Information")
print(f"raw at 0: {raw_docs[0]}")
print(len(raw_docs))
st.text(f"Raw Response 0: {raw_docs[0]}")
st.text(f"Raw Response 1: {raw_docs[1]}")
st.text(f"Raw Response 2: {raw_docs[2]}")
st.text(f"Raw Response 3: {raw_docs[3]}")
st.text(f"Raw Response 4: {raw_docs[4]}")


st.header("RAG Response")
st.write(rag_response)